<a href="https://colab.research.google.com/github/99kenny/VGGNet-for-cifar-10/blob/main/ResNet_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Activation, ReLU, GlobalAveragePooling2D, Add
from keras.models import Model
from keras import optimizers, regularizers, utils
from keras.preprocessing.image import ImageDataGenerator
from keras import utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2

In [19]:
# download data
cifar10 = tf.keras.datasets.cifar10 
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog','frog', 'horse', 'ship', 'truck']

In [20]:
mean = np.mean(train_images,axis=(0,1,2,3))
std = np.std(train_images, axis=(0, 1, 2, 3))
train_images = (train_images - mean)/(std+1e-7)
test_images = (test_images - mean)/(std+1e-7)

train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,
            )  # randomly flip images

datagen.fit(train_images)

In [21]:
weight_decay = [1e-4]
wd = 1e-4
dropout = 0
lr = 1e-3
batch_size = 256
hists = []

In [22]:
n = 9 # 56 layers
channels = [16, 32, 64]
inputs = Input(shape=(32, 32, 3))

def resNet():
  x = Conv2D(channels[0], kernel_size=(3, 3), padding="same", kernel_initializer="he_normal", kernel_regularizer=l2(wd))(inputs)
  x = BatchNormalization()(x)
  x = Activation(tf.nn.relu)(x)

  for c in channels:
      for i in range(n):
          subsampling = (i == 0 and c > 16)
          strides = (2, 2) if subsampling else (1, 1)
          y = Conv2D(c, kernel_size=(3, 3), padding="same", strides=strides, kernel_initializer="he_normal", kernel_regularizer=l2(wd))(x)
          y = BatchNormalization()(y)
          y = Activation(tf.nn.relu)(y)
          y = Conv2D(c, kernel_size=(3, 3), padding="same", kernel_initializer="he_normal", kernel_regularizer=l2(wd))(y)
          y = BatchNormalization()(y)    
          y = Dropout(dropout)(y)    
          if subsampling:
              x = Conv2D(c, kernel_size=(1, 1), strides=(2, 2), padding="same", kernel_initializer="he_normal", kernel_regularizer=l2(wd))(x)
          x = Add()([x, y])
          x = Activation(tf.nn.relu)(x)

  x = GlobalAveragePooling2D()(x)
  x = Flatten()(x)
  outputs = Dense(10, activation=tf.nn.softmax, kernel_initializer="he_normal")(x)
  return Model(inputs=inputs, outputs=outputs)

In [ ]:
for batch_size in [128,256]:
  model = resNet()
  model.compile(optimizer='Adam', 
                  loss='categorical_crossentropy', metrics=['accuracy'])


  from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
  learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                              patience=10, 
                                              verbose=1, 
                                              factor=0.1, 
                                              min_lr=0.000000001)

  hists.append(model.fit(datagen.flow(train_images, train_labels, batch_size=256), epochs=50,
              validation_data=(test_images, test_labels),callbacks = [learning_rate_reduction]))

Epoch 1/50
196/196 [==============================] - 66s 142ms/step - loss: 2.2275 - accuracy: 0.3773 - val_loss: 3.0776 - val_accuracy: 0.2570 - lr: 0.0010
Epoch 2/50
196/196 [==============================] - 27s 135ms/step - loss: 1.6760 - accuracy: 0.5367 - val_loss: 2.8689 - val_accuracy: 0.3648 - lr: 0.0010
Epoch 3/50
196/196 [==============================] - 26s 134ms/step - loss: 1.4489 - accuracy: 0.6167 - val_loss: 1.7303 - val_accuracy: 0.5674 - lr: 0.0010
Epoch 4/50
196/196 [==============================] - 26s 134ms/step - loss: 1.2827 - accuracy: 0.6763 - val_loss: 1.4767 - val_accuracy: 0.6241 - lr: 0.0010
Epoch 5/50
196/196 [==============================] - 26s 132ms/step - loss: 1.1712 - accuracy: 0.7095 - val_loss: 1.4830 - val_accuracy: 0.6356 - lr: 0.0010
Epoch 6/50
196/196 [==============================] - 26s 135ms/step - loss: 1.0790 - accuracy: 0.7401 - val_loss: 1.5333 - val_accuracy: 0.6174 - lr: 0.0010
Epoch 7/50
196/196 [==============================] 

In [ ]:
for hist in hists:
  fig, acc_ax = plt.subplots()


  acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
  acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
  acc_ax.set_ylabel('accuracy')
  acc_ax.legend(loc='upper left')

  plt.show()

  outputs = model.predict(test_images)

  e1 = tf.keras.metrics.categorical_accuracy(
      test_labels, outputs
  )

  e2 = tf.keras.metrics.top_k_categorical_accuracy(
      test_labels, outputs, k=5
  )

  print("top-1 error: ", (np.count_nonzero(e1 == 0) / 10000))
  print("top-5 error: ", (np.count_nonzero(e2 == 0) / 10000))